# Лабораторная работа №2

## Криптоанализ симметричных криптосистем

**Задание:**

Провести эксперимент по определению практической стойкости, алгоритма, разработанного в лабораторной работе №1. 

Считать, что противнику известен алгоритм шифрования. Выбрать наилучший с его точки зрения алгоритм подбора ключа и обосновать свой выбор. Использовать методы:
- анализа статистических свойств шифртекста (частот появления букв).
- силовую атаку (полный перебор ключей).
- другие (если есть более эффективные)

С помощью программы, реализующей выбранный алгоритм крип-тоана¬лиза провести экперимент по вскрытию шифртекс¬тов различного размера.

При использовании статистического криптоанализа исполь¬зовать табли¬цы, приведенные в приложении или подсчитать частоты появления букв используемого алфавита в тексте, частью которого является текст примера.

Для проверки на «осмысленность» полученного текста создать мини-словарь из части слов, встречающихся в тексте примера. 
Построить графики зависимости времени криптоанализа от параметров ал¬горитма шифрования (длины или других параметров ключа, размера шифр¬текста или др., в зависимости от алгоритмов шифрования и криптоанализа).

В результате эксперимента определить параметры алгоритма шифрования (размер передаваемого текста, размер и характеристи¬ки ключа, объем ключевого пространства и другие параметры алгоритма шифрования), необходимые для практической криптостойкости разработанного в лабораторной работе №1 алгоритма ширования. 

Практической криптостойкостью в данной работе будем считать невозможность взлома шифра противником, имеющим в распоряжении один ПК мощности, равной мощности компьютера, на котором делалась работа и один час времени.

### Выбор метода дешифрования

В лабораторной работе №1 был использован табличный шифр с перестановкой по ключу. Так как мы имеем дело с перестановочным шифром, а не шифром, связанным с заменой, то частотный анализ нам не подойдет, так как при перестановке частота появления символов не меняется

Используем в данной работе силовую атаку (полный перебор ключей). По числу столбцов таблицы мы можем определить длину ключа и далее провести перебор ключей этой длины. Вдобавок, мы будем знать алфавит шифртекста - заглавные латинские буквы, что должно облегчить работу "взломщика"

Если алгоритм-взломщик не справится с дешифровкой за 1 час - будем считать шифр достаточно криптостойким

### Импорт необходимых библиотек

In [1]:
from core.utils.common import *
from core.utils.file_utils import *
from core.text.hack import CipherBreaker

Для воспроизводимости результатов

In [2]:
init_random_seed(316)

Определим пути к различным файлам, согласно заданию

In [3]:
data_path = 'data'

short_text_encrypted_path = f"{data_path}/encrypted/short-text-encrypted.txt"
short_text_decrypted_path = f"{data_path}/decrypted/short-text-decrypted.txt"

long_text_encrypted_path = f"{data_path}/encrypted/long-text-encrypted.txt"
long_text_decrypted_path = f"{data_path}/decrypted/long-text-decrypted.txt"

### Взлом зашифрованного короткого примера

Текст, который должен получиться

In [11]:
short_text = read_plain_text_from_file(short_text_decrypted_path)
print(short_text)

для чего тюленю так много сил? чтобы запомнить, что тюлень по-английски seal


Построим валидатор - набор случайных слов из целевого текста. Если в результате взлома полученный текст будет содержать все слова из валидатора - взлом будем считать успешным

In [5]:
short_text_validator = generate_validation_dictionary(short_text)
short_text_validator

{'seal',
 'для',
 'запомнить',
 'много',
 'по-английски',
 'сил',
 'так',
 'тюлень',
 'тюленю',
 'чего',
 'что',
 'чтобы'}

Зашифрованный короткий текстовый пример

In [6]:
short_text_encrypted = read_plain_text_from_file(short_text_encrypted_path)
print(short_text_encrypted)

дтя лгчо е
ю еклтюан 
м?олнсоиг 
 птач бзоы
отнчм,т иь
опт  нльюе
окас-игйнл
и s   a el


In [7]:
small_text_breaker = CipherBreaker(short_text_encrypted_path, validator=short_text_validator)

Шифр-текст успешно загружен:
| д | т | я |   | л | г | ч | о |   | е |
-----------------------------------------
| ю |   | е | к | л | т | ю | а | н |   |
-----------------------------------------
| м | ? | о | л | н | с | о | и | г |   |
-----------------------------------------
|   | п | т | а | ч |   | б | з | о | ы |
-----------------------------------------
| о | т | н | ч | м | , | т |   | и | ь |
-----------------------------------------
| о | п | т |   |   | н | л | ь | ю | е |
-----------------------------------------
| о | к | а | с | - | и | г | й | н | л |
-----------------------------------------
| и |   | s |   |   |   | a |   | e | l |
-----------------------------------------


Перебор ключей с таймаутом по умолачанию - 1 час

In [8]:
try:
    small_text_decrypted = small_text_breaker.hack_by_key_brute_force()
    print(small_text_decrypted)
except TimeoutError:
    print("Взлом шифра методом перебора ключей не увенчался успехом")

Не получилось взломать шифр за отведенное время 3600 сек. Было совершено попыток 79079414
Взлом шифра методом перебора ключей не увенчался успехом


Короткий текстовый пример не был расшифрован за 1 час перебора ключей, за это время алгоритм принял `79079414` попыток, что говорит о криптостойкости алгоритма шифрования

### Взлом зашифрованного длинного примера

Текст, который должен получиться

In [10]:
long_text = read_plain_text_from_file(long_text_decrypted_path)
print(long_text)

[интро]
patrzcie co spotkałem

[припев]
ej! bóbr, kurwa!
ej! bóbr
ej! bóbr, kurwa!
ej! bóbr

[куплет 1]
сидим с бобром за столом
вдвоём, на ужин готовим полено
давай поговорим, бобёр
о том, что наболело
скажи, зачем же между нами плотина?
скажи, зачем между нами обрыв?
я обниму твоё пушистое тело
ну почему бобры так добры?

[куплет 2]
сидим с бобром за столом
я и он, текила, ром, точим полено
скажи мне честно, бобёр
почему, бобёр, мне так офигенно?

[припев]
ej! bóbr, kurwa (будь добр, будь добр!)
ej! bóbr, bóbr (бобр, будь добр, будь kurwa!)
ej! bóbr, bóbr (будь добр, будь добр!)
ej! bóbr, bóbr (бобр, будь добр, будь kurwa!)

[куплет 3]
идём с бобром мы в лесу
на руках его несу (болтает хвост на весу)
идём вдвоём и поём
с бобром хиты алсу (алсу)

[припев]
ej! bóbr, kurwa (будь добр, будь добр!)
ej! bóbr, bóbr (бобр, будь добр, будь kurwa!)
ej! bóbr, bóbr (будь добр, будь добр!)
ej! bóbr, bóbr (бобр, будь добр, будь kurwa!)

[аутро]
пришли мы к дому бобра (бобра)
он мне дал (дал) с соб

Построим валидатор - набор случайных слов из целевого текста. Если в результате взлома полученный текст будет содержать все слова из валидатора - взлом будем считать успешным

In [14]:
long_text_validator = generate_validation_dictionary(long_text, words_frac=0.5)
long_text_validator

{'2',
 'bóbr',
 'co',
 'ej',
 'kurwa',
 'patrzcie',
 'а-а',
 'алсу',
 'аутро',
 'бобр',
 'бобра',
 'бобром',
 'бобёр',
 'болтает',
 'будь',
 'в',
 'вдвоём',
 'весу',
 'всё',
 'добр',
 'добра',
 'добры',
 'его',
 'же',
 'за',
 'зачем',
 'и',
 'к',
 'куплет',
 'лесу',
 'между',
 'мы',
 'на',
 'наболело',
 'несу',
 'ну',
 'о',
 'обниму',
 'он',
 'офигенно',
 'полено',
 'попёр',
 'почему',
 'припев',
 'ром',
 'руках',
 'с',
 'скажи',
 'собой',
 'столом',
 'так',
 'твоё',
 'текила',
 'ужин',
 'хиты',
 'честно',
 'что',
 'я'}

Зашифрованный длинный текстовый пример

In [15]:
long_text_encrypted = read_plain_text_from_file(long_text_encrypted_path)
print(long_text_encrypted)

и]⏎pрa[нот
r coi tzec
płemk⏎soat
[ев]и⏎⏎ппр
jbr,b e!ó 
u⏎eja!kr!w
bej!r  ó⏎b
ókur,wbb r
! bójba⏎!e
⏎плектr⏎у[
1димс  ]и⏎
 ом бзсбро
 ом⏎оваслт
в нам до,ё
жото вуигн
меноо⏎и лп
апогйодв а
о бомбвр,и
ром,  ё⏎то
тболнечоа 
ожи,к л⏎ас
аже ммзч е
жами  едну
ла?⏎испонт
азач,екж и
 у нжаммде
иыв?б⏎м ро
 му нтяоиб
ошисптвёу 
ео⏎неуо лт
пу бео омч
рк дтобыа 
р[ку⏎пбы⏎?
е⏎си2длт] 
мобр ои бс
 тол омзса
⏎он,и мя  
е, рлоткаи
,им опм чт
лскаожое⏎н
 чесетим н
обёрб⏎н,о 
о, бмопчуе
ёне  тбрм,
кгенфна ио
?рип[ео⏎п⏎
] bójbв⏎!e
,wa u(r rk
уобр ,бддь
бдобьр у д
) bójb!⏎!e
,r (óбr bb
будь  орб,
обуд,ьдб р
k!)⏎we uar
!r, óbj bb
bудь( órб 
обуд,ьдб р
д)⏎eрj о!б
 , bbó!bró
rбр,б b о(
уобр ,бддь
бkurьw у д
!куп⏎лa)[⏎
тидё]ме ⏎3
сромо   бб
ыесу ⏎м лв
аах уен кр
оу (ебг сн
л хвеоотта
твесаус  н
⏎ вдёв)имд
ёпоёимом  
сромо ⏎ бб
илсу  хтаы
а⏎⏎[уп(л)с
и⏎ejв!рп]е
b kurr ó,b
aдь бдw у(
будь  орб,
о⏎ej!!дб)р
b bórb ó,b
 р, обr(бб
дбр,д уьо 
уurw aбдkь
) bójb!⏎!e
,r (óбr bb
дбр,д уьо 
уобр !бддь
⏎bób!r)e j
  (бbо,bró

In [16]:
long_text_breaker = CipherBreaker(long_text_encrypted_path, validator=long_text_validator)

Шифр-текст успешно загружен:
| и | ] | ⏎ | p | р | a | [ | н | о | т |
-----------------------------------------
| r |   | c | o | i |   | t | z | e | c |
-----------------------------------------
| p | ł | e | m | k | ⏎ | s | o | a | t |
-----------------------------------------
| [ | е | в | ] | и | ⏎ | ⏎ | п | п | р |
-----------------------------------------
| j | b | r | , | b |   | e | ! | ó |   |
-----------------------------------------
| u | ⏎ | e | j | a | ! | k | r | ! | w |
-----------------------------------------
| b | e | j | ! | r |   |   | ó | ⏎ | b |
-----------------------------------------
| ó | k | u | r | , | w | b | b |   | r |
-----------------------------------------
| ! |   | b | ó | j | b | a | ⏎ | ! | e |
-----------------------------------------
| ⏎ | п | л | е | к | т | r | ⏎ | у | [ |
-----------------------------------------
| 1 | д | и | м | с |   |   | ] | и | ⏎ |
-----------------------------------------
|   | о | м |   | б | з | с | б | р | о |
-----

Перебор ключей с таймаутом по умолачанию - 1 час

In [17]:
try:
    long_text_decrypted = long_text_breaker.hack_by_key_brute_force()
    print(long_text_decrypted)
except TimeoutError:
    print("Взлом шифра методом перебора ключей не увенчался успехом")

Не получилось взломать шифр за отведенное время 3600 сек. Было совершено попыток 9765910
Взлом шифра методом перебора ключей не увенчался успехом


Длинный текстовый пример не был расшифрован за 1 час перебора ключей, за это время алгоритм принял `9765910` попыток, что говорит о криптостойкости алгоритма шифрования

Число попыток, затраченных на взлом длинного текстового примера был на порядок меньше чем при взломе короткого, что логично. В обоих случаях взлом полным перебором ключей не увенчался успехом, что говорит о криптостойкости алгоритма шифрования